In [1]:
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
## All tiles of Bangladesh

N20E92 = ee.Geometry.Polygon(
        [[[92, 21],
          [93, 21],
          [93, 20],
          [92, 20],
          [92, 21]]]) 
N21E89 = ee.Geometry.Polygon(
        [[[89 22],
          [90,22],
          [90, 21],
          [89, 21],
          [89, 22]]]) 

N21E90 = ee.Geometry.Polygon(
        [[[90, 22],
          [91, 22],
          [91, 21],
          [90, 21],
          [90, 22]]])

N21E91 = ee.Geometry.Polygon(
        [[[91, 22],
          [92, 22],
          [92, 21],
          [91, 21],
          [91, 22]]])
N21E92 = ee.Geometry.Polygon(
        [[[92, 22],
          [93, 22],
          [93, 21],
          [92, 21],
          [92, 22]]])
N22E88 = ee.Geometry.Polygon(
        [[[88, 23],
          [89, 23],
          [89, 22],
          [88, 22],
          [88, 23]]])
N22E89 = ee.Geometry.Polygon(
        [[[89 23],
          [90,23],
          [90, 22],
          [89, 22],
          [89, 23]]])
N22E90 = ee.Geometry.Polygon(
        [[[90, 23],
          [91, 23],
          [91, 22],
          [90, 22],
          [90, 23]]])
N22E91 = ee.Geometry.Polygon(
        [[[91, 23],
          [92, 23],
          [92, 22],
          [91, 22],
          [91, 23]]])
N22E92 = ee.Geometry.Polygon(
        [[[92, 23],
          [93, 23],
          [93, 22],
          [92, 22],
          [92, 23]]])
N23E88 = ee.Geometry.Polygon(
        [[[88, 24],
          [89, 24],
          [89, 23],
          [88, 23],
          [88, 24]]])
N23E89 = ee.Geometry.Polygon(
        [[[89 24],
          [90,24],
          [90, 23],
          [89, 23],
          [89, 24]]])
N23E90 = ee.Geometry.Polygon(
        [[[90, 24],
          [91, 24],
          [91, 23],
          [90, 23],
          [90, 24]]])
N23E91 = ee.Geometry.Polygon(
        [[[91, 24],
          [92, 24],
          [92, 23],
          [91, 23],
          [91, 24]]])

N23E92 = ee.Geometry.Polygon(
        [[[92, 24],
          [93, 24],
          [93, 23],
          [92, 23],
          [92, 24]]])

N24E88 = ee.Geometry.Polygon(
        [[[88, 25],
          [89, 25],
          [89, 24],
          [88, 24],
          [88, 25]]])
N24E89 = ee.Geometry.Polygon(
        [[[89, 25],
          [90, 25],
          [90, 24],
          [89, 24],
          [89, 25]]])
N24E90 = ee.Geometry.Polygon(
        [[[90, 25],
          [91, 25],
          [91, 24],
          [90, 24],
          [90, 25]]])
N24E91 = ee.Geometry.Polygon(
        [[[91, 25],
          [92, 25],
          [92, 24],
          [91, 24],
          [91, 25]]])
N24E92 = ee.Geometry.Polygon(
        [[[92, 25],
          [93, 25],
          [93, 24],
          [92, 24],
          [92, 25]]])
N25E88 = ee.Geometry.Polygon(
        [[[88, 26],
          [89, 26],
          [89, 25],
          [88, 25],
          [88, 26]]])
N25E89 = ee.Geometry.Polygon(
        [[[89, 26],
          [90, 26],
          [90, 25],
          [89, 25],
          [89, 26]]])
N25E90 = ee.Geometry.Polygon(
        [[[90, 25],
          [91, 25],
          [91, 26],
          [90, 26],
          [90, 25]]])
N25E91 = ee.Geometry.Polygon(
        [[[91, 26],
          [92, 26],
          [92, 25],
          [91, 25],
          [91, 26]]])
N25E92 = ee.Geometry.Polygon(
        [[[92, 26],
          [93, 26],
          [93, 25],
          [92, 25],
          [92, 26]]])

N26E88 = ee.Geometry.Polygon(
        [[[88, 27],
          [89, 27],
          [89, 26],
          [88, 26],
          [88, 27]]])

N26E89 = ee.Geometry.Polygon(
        [[[89, 27],
          [90, 27],
          [90, 26],
          [89, 26],
          [89, 27]]])

In [ ]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.   
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))
        
    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [ ]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [ ]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
#     not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
#     dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [ ]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
#     not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
#     dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [ ]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [ ]:
# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

In [ ]:
def fill_nodata(img_t0, img_t1):
    mask_t0 = img_t0.unmask(0)
    mask_t1 = img_t1.unmask(0)    
    t0_filled = mask_t0.where(mask_t0.eq(0), img_t1)
    return ee.Image(t0_filled)

In [ ]:
# list of region of interest

aoi = [N20E92,N21E89,N21E90,N21E91,N21E92,N22E88,N22E89,N22E90,N22E91,
       N22E92,N23E88,N23E89,N23E90,N23E91,N23E92,N24E88,N24E89,N24E90,
       N24E91,N24E92,N25E88,N25E89,N25E90,N25E91,N25E92,N26E88,N26E89 ]

In [ ]:
# define the period for searching data
year1 = '2022'
year2 = '2023'
year3 = '2024'

# year 1
START_y1 = ee.Date(year1 + '-01-01');
END_y1 = ee.Date(year1 + '-12-31');
start01_y1 = ee.Date(year1 + '-01-01');
end01_y1 = ee.Date(year1 + '-01-31');
start02_y1 = ee.Date(year1 + '-02-01');
end02_y1 = ee.Date(year1 + '-02-28');
start03_y1 = ee.Date(year1 + '-03-01');
end03_y1 = ee.Date(year1 + '-03-31');
start04_y1 = ee.Date(year1 + '-04-01');
end04_y1 = ee.Date(year1 + '-04-30');
start05_y1 = ee.Date(year1 + '-05-01');
end05_y1 = ee.Date(year1 + '-05-31');
start06_y1 = ee.Date(year1 + '-06-01');
end06_y1 = ee.Date(year1 + '-06-30');
start07_y1 = ee.Date(year1 + '-07-01');
end07_y1 = ee.Date(year1 + '-07-31');
start08_y1 = ee.Date(year1 + '-08-01');
end08_y1 = ee.Date(year1 + '-08-31');
start09_y1 = ee.Date(year1 + '-09-01');
end09_y1 = ee.Date(year1 + '-09-30');
start10_y1 = ee.Date(year1 + '-10-01');
end10_y1 = ee.Date(year1 + '-10-31');
start11_y1 = ee.Date(year1 + '-11-01');
end11_y1 = ee.Date(year1 + '-11-30');
start12_y1 = ee.Date(year1 + '-12-01');
end12_y1 = ee.Date(year1 + '-12-31');

# year 2
START_y2 = ee.Date(year2 + '-01-01');
END_y2 = ee.Date(year2 + '-12-31');
start01_y2 = ee.Date(year2 + '-01-01');
end01_y2 = ee.Date(year2 + '-01-31');
start02_y2 = ee.Date(year2 + '-02-01');
end02_y2 = ee.Date(year2 + '-02-28');
start03_y2 = ee.Date(year2 + '-03-01');
end03_y2 = ee.Date(year2 + '-03-31');
start04_y2 = ee.Date(year2 + '-04-01');
end04_y2 = ee.Date(year2 + '-04-30');
start05_y2 = ee.Date(year2 + '-05-01');
end05_y2 = ee.Date(year2 + '-05-31');
start06_y2 = ee.Date(year2 + '-06-01');
end06_y2 = ee.Date(year2 + '-06-30');
start07_y2 = ee.Date(year2 + '-07-01');
end07_y2 = ee.Date(year2 + '-07-31');
start08_y2 = ee.Date(year2 + '-08-01');
end08_y2 = ee.Date(year2 + '-08-31');
start09_y2 = ee.Date(year2 + '-09-01');
end09_y2 = ee.Date(year2 + '-09-30');
start10_y2 = ee.Date(year2 + '-10-01');
end10_y2 = ee.Date(year2 + '-10-31');
start11_y2 = ee.Date(year2 + '-11-01');
end11_y2 = ee.Date(year2 + '-11-30');
start12_y2 = ee.Date(year2 + '-12-01');
end12_y2 = ee.Date(year2 + '-12-31');

# year 3
START_y3 = ee.Date(year3 + '-01-01');
END_y3 = ee.Date(year3 + '-12-31');
start01_y3 = ee.Date(year3 + '-01-01');
end01_y3 = ee.Date(year3 + '-01-31');
start02_y3 = ee.Date(year3 + '-02-01');
end02_y3 = ee.Date(year3 + '-02-28');
start03_y3 = ee.Date(year3 + '-03-01');
end03_y3 = ee.Date(year3 + '-03-31');
start04_y3 = ee.Date(year3 + '-04-01');
end04_y3 = ee.Date(year3 + '-04-30');
start05_y3 = ee.Date(year3 + '-05-01');
end05_y3 = ee.Date(year3 + '-05-31');
start06_y3 = ee.Date(year3 + '-06-01');
end06_y3 = ee.Date(year3 + '-06-30');
start07_y3 = ee.Date(year3 + '-07-01');
end07_y3 = ee.Date(year3 + '-07-31');
start08_y3 = ee.Date(year3 + '-08-01');
end08_y3 = ee.Date(year3 + '-08-31');
start09_y3 = ee.Date(year3 + '-09-01');
end09_y3 = ee.Date(year3 + '-09-30');
start10_y3 = ee.Date(year3 + '-10-01');
end10_y3 = ee.Date(year3 + '-10-31');
start11_y3 = ee.Date(year3 + '-11-01');
end11_y3 = ee.Date(year3 + '-11-30');
start12_y3 = ee.Date(year3 + '-12-01');
end12_y3 = ee.Date(year3 + '-12-31');

In [ ]:
starts_y1 = [start01_y1, start02_y1, start03_y1, start04_y1, start05_y1, start06_y1, start07_y1, start08_y1, start09_y1, start10_y1, start11_y1, start12_y1]
starts_y2 = [start01_y2, start02_y2, start03_y2, start04_y2, start05_y2, start06_y2, start07_y2, start08_y2, start09_y2, start10_y2, start11_y2, start12_y2]
starts_y3 = [start01_y3, start02_y3, start03_y3, start04_y3, start05_y3, start06_y3, start07_y3, start08_y3, start09_y3, start10_y3, start11_y3, start12_y3]

In [ ]:
ends_y1 = [end01_y1, end02_y1, end03_y1, end04_y1, end05_y1, end06_y1, end07_y1, end08_y1, end09_y1, end10_y1, end11_y1, end12_y1]
ends_y2 = [end01_y2, end02_y2, end03_y2, end04_y2, end05_y2, end06_y2, end07_y2, end08_y2, end09_y2, end10_y2, end11_y2, end12_y2]
ends_y3 = [end01_y3, end02_y3, end03_y3, end04_y3, end05_y3, end06_y3, end07_y3, end08_y3, end09_y3, end10_y3, end11_y3, end12_y3]

In [ ]:
months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

In [ ]:

tile = ["N20E92", "N21E89", "N21E90", "N21E91", "N21E92", "N22E88", "N22E89", "N22E90", "N22E91", "N22E92",
    "N23E88", "N23E89", "N23E90", "N23E91", "N23E92", "N24E88", "N24E89", "N24E90", "N24E91", "N24E92",
    "N25E88", "N25E89", "N25E90", "N25E91", "N25E92", "N26E88", "N26E89" ]


In [ ]:
import os
import geemap
from googleapiclient import errors
from IPython.display import Image


In [ ]:
pip install geedim

In [ ]:
# median composite image using new approach of images from 2019,2020,2021
# parameters list
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100
CLOUD_FILTER = 30
CLD_PRB_THRESH = 60
for i in range(0,60):
    imgListy1 = []
    imgListy2 = []
    imgListy3 = []
    for j in range(0,12):       
        #select images for year 1
        s2_sr_cld_col_y1 = get_s2_sr_cld_col(aoi[i], starts_y1[j], ends_y1[j])
        s2_sr_add_mask_y1 = s2_sr_cld_col_y1.map(add_cld_shdw_mask)
        s2_sr_mask_y1 = s2_sr_add_mask_y1.map(apply_cld_shdw_mask)
        imgListy1.append(s2_sr_mask_y1)
        
        #select images for year 2
        s2_sr_cld_col_y2 = get_s2_sr_cld_col(aoi[i], starts_y2[j], ends_y2[j])
        s2_sr_add_mask_y2 = s2_sr_cld_col_y2.map(add_cld_shdw_mask)
        s2_sr_mask_y2 = s2_sr_add_mask_y2.map(apply_cld_shdw_mask) 
        imgListy2.append(s2_sr_mask_y2)
        
        #select images for year 3
        s2_sr_cld_col_y3 = get_s2_sr_cld_col(aoi[i], starts_y3[j], ends_y3[j])
        s2_sr_add_mask_y3 = s2_sr_cld_col_y3.map(add_cld_shdw_mask)
        s2_sr_mask_y3 = s2_sr_add_mask_y3.map(apply_cld_shdw_mask)
        imgListy3.append(s2_sr_mask_y3)
       
    img_01_y2 = imgListy2[0].merge(imgListy2[0]).merge(imgListy2[0]).merge(imgListy2[1]).merge(imgListy2[1]).merge(imgListy2[1]).merge(imgListy2[2]).merge(imgListy1[0]).merge(imgListy1[1]).merge(imgListy3[0]).merge(imgListy3[1]).median()
    img_03_y2 = imgListy2[2].merge(imgListy2[2]).merge(imgListy2[2]).merge(imgListy2[3]).merge(imgListy2[3]).merge(imgListy2[3]).merge(imgListy2[1]).merge(imgListy2[4]).merge(imgListy1[2]).merge(imgListy1[3]).merge(imgListy3[2]).merge(imgListy3[3]).median()
    img_05_y2 = imgListy2[4].merge(imgListy2[4]).merge(imgListy2[4]).merge(imgListy2[5]).merge(imgListy2[5]).merge(imgListy2[5]).merge(imgListy2[3]).merge(imgListy2[6]).merge(imgListy1[3]).merge(imgListy1[4]).merge(imgListy1[5]).merge(imgListy1[6]).merge(imgListy3[3]).merge(imgListy3[4]).merge(imgListy3[5]).merge(imgListy3[6]).median()
    img_07_y2 = imgListy2[6].merge(imgListy2[6]).merge(imgListy2[6]).merge(imgListy2[7]).merge(imgListy2[7]).merge(imgListy2[7]).merge(imgListy2[5]).merge(imgListy2[8]).merge(imgListy1[5]).merge(imgListy1[6]).merge(imgListy1[7]).merge(imgListy1[8]).merge(imgListy3[5]).merge(imgListy3[6]).merge(imgListy3[7]).merge(imgListy3[8]).median()
    img_09_y2 = imgListy2[8].merge(imgListy2[8]).merge(imgListy2[8]).merge(imgListy2[9]).merge(imgListy2[9]).merge(imgListy2[9]).merge(imgListy2[7]).merge(imgListy2[10]).merge(imgListy1[7]).merge(imgListy1[8]).merge(imgListy1[9]).merge(imgListy1[10]).merge(imgListy3[7]).merge(imgListy3[8]).merge(imgListy3[9]).merge(imgListy3[10]).median()
    img_11_y2 = imgListy2[10].merge(imgListy2[10]).merge(imgListy2[10]).merge(imgListy2[11]).merge(imgListy2[11]).merge(imgListy2[11]).merge(imgListy2[9]).merge(imgListy1[10]).merge(imgListy1[11]).merge(imgListy3[10]).merge(imgListy3[11]).median()
       
    
    img_01_10m_y2 = img_01_y2.select(['B2', 'B3', 'B4', 'B8'])
    img_01_20m_y2 = img_01_y2.select(['B5', 'B6', 'B7', 'B8A', 'B11', 'B12'])
    
    img_03_10m_y2 = img_03_y2.select(['B2', 'B3', 'B4', 'B8'])
    img_03_20m_y2 = img_03_y2.select(['B5', 'B6', 'B7', 'B8A', 'B11', 'B12'])
    
    img_05_10m_y2 = img_05_y2.select(['B2', 'B3', 'B4', 'B8'])
    img_05_20m_y2 = img_05_y2.select(['B5', 'B6', 'B7', 'B8A', 'B11', 'B12'])
    
    img_07_10m_y2 = img_07_y2.select(['B2', 'B3', 'B4', 'B8'])
    img_07_20m_y2 = img_07_y2.select(['B5', 'B6', 'B7', 'B8A', 'B11', 'B12'])
    
    img_09_10m_y2 = img_09_y2.select(['B2', 'B3', 'B4', 'B8'])
    img_09_20m_y2 = img_09_y2.select(['B5', 'B6', 'B7', 'B8A', 'B11', 'B12'])
    
    img_11_10m_y2 = img_11_y2.select(['B2', 'B3', 'B4', 'B8'])
    img_11_20m_y2 = img_11_y2.select(['B5', 'B6', 'B7', 'B8A', 'B11', 'B12'])
    
    
    # create path to save image

    path_10m = os.getcwd() + '/2022/sentinel2_10m_2023/' + tile[i] + '/'
    path_20m = os.getcwd() + '/2022/sentinel2_20m_2023/' + tile[i] + '/'
    if not os.path.exists(os.path.join(path_10m)):
        os.makedirs(os.path.join(path_10m))
    if not os.path.exists(os.path.join(path_20m)):
        os.makedirs(os.path.join(path_20m))
    
    name_10m_01 = tile[i] + '_10m_01-01_median.tif'
    name_10m_03 = tile[i] + '_10m_03-01_median.tif'
    name_10m_05 = tile[i] + '_10m_05-01_median.tif'
    name_10m_07 = tile[i] + '_10m_07-01_median.tif'
    name_10m_09 = tile[i] + '_10m_09-01_median.tif'
    name_10m_11 = tile[i] + '_10m_11-01_median.tif'
    
    name_20m_01 = tile[i] + '_20m_01-01_median.tif'                      
    name_20m_03 = tile[i] + '_20m_03-01_median.tif'                      
    name_20m_05 = tile[i] + '_20m_05-01_median.tif'                      
    name_20m_07 = tile[i] + '_20m_07-01_median.tif'                      
    name_20m_09 = tile[i] + '_20m_09-01_median.tif'                      
    name_20m_11 = tile[i] + '_20m_11-01_median.tif'                      
    
    name_10m_01 = path_10m + name_10m_01
    name_10m_03 = path_10m + name_10m_03
    name_10m_05 = path_10m + name_10m_05
    name_10m_07 = path_10m + name_10m_07
    name_10m_09 = path_10m + name_10m_09
    name_10m_11 = path_10m + name_10m_11
    
    name_20m_01 = path_20m + name_20m_01
    name_20m_03 = path_20m + name_20m_03
    name_20m_05 = path_20m + name_20m_05
    name_20m_07 = path_20m + name_20m_07
    name_20m_09 = path_20m + name_20m_09
    name_20m_11 = path_20m + name_20m_11
    
    geemap.download_ee_image(img_01_10m_y2, name_10m_01, region=aoi[i], crs="EPSG:4326", scale=10)
    geemap.download_ee_image(img_03_10m_y2, name_10m_03, region=aoi[i], crs="EPSG:4326", scale=10)
    geemap.download_ee_image(img_05_10m_y2, name_10m_05, region=aoi[i], crs="EPSG:4326", scale=10)
    geemap.download_ee_image(img_07_10m_y2, name_10m_07, region=aoi[i], crs="EPSG:4326", scale=10)
    geemap.download_ee_image(img_09_10m_y2, name_10m_09, region=aoi[i], crs="EPSG:4326", scale=10)
    geemap.download_ee_image(img_11_10m_y2, name_10m_11, region=aoi[i], crs="EPSG:4326", scale=10)
    
    geemap.download_ee_image(img_01_20m_y2, name_20m_01, region=aoi[i], crs="EPSG:4326", scale=20)
    geemap.download_ee_image(img_03_20m_y2, name_20m_03, region=aoi[i], crs="EPSG:4326", scale=20)
    geemap.download_ee_image(img_05_20m_y2, name_20m_05, region=aoi[i], crs="EPSG:4326", scale=20)
    geemap.download_ee_image(img_07_20m_y2, name_20m_07, region=aoi[i], crs="EPSG:4326", scale=20)
    geemap.download_ee_image(img_09_20m_y2, name_20m_09, region=aoi[i], crs="EPSG:4326", scale=20)
    geemap.download_ee_image(img_11_20m_y2, name_20m_11, region=aoi[i], crs="EPSG:4326", scale=20)
    